In [ ]:
%load_ext autoreload

%autoreload 2


In [ ]:
import os,sys
os.environ['BASE_DIR'] =  '/Users/macico/Dropbox/btc'
os.environ['KULOKO_DIR'] = os.path.join(os.environ['BASE_DIR'], "kuloko")
os.environ['ALEISTER_DIR'] = os.path.join(os.environ['BASE_DIR'], "aleister")
os.environ['COMMON_DIR'] = os.path.join(os.environ['BASE_DIR'], "geco_commons")
os.environ['KULOKO_INI'] = os.path.join(os.environ['COMMON_DIR'], "ini")
os.environ['ALEISTER_INI'] = os.path.join(os.environ['COMMON_DIR'], "ini")
os.environ['MONGO_DIR'] = os.path.join(os.environ['COMMON_DIR'] ,"mongodb")
os.environ['LOGDIR'] = os.path.join(os.environ['KULOKO_DIR'], "log")
# os.environ['MLFLOW_TRACKING_URI'] = os.path.join(os.environ['BASE_DIR'], "mlruns")
# OS.environ['MLFLOW_ARTIFACTS_URI'] = os.path.join(os.environ['BASE_DIR'], "mlruns")
os.environ['MLFLOW_TRACKING_URI'] = 'http://localhost:5000'
os.environ['MLFLOW_ARTIFACTS_URI'] = os.path.join(os.environ['BASE_DIR'], "mlruns")
os.environ['DEFAULT_LOCAL_FILE_AND_ARTIFACT_PATH'] = os.path.join(os.environ['BASE_DIR'], "mlruns")
sys.path.append(os.path.join(os.environ['KULOKO_DIR'],"items" ))


In [ ]:


import master
from master import *
from datetime import datetime as dt

In [ ]:
# args=['--execute_mode',"prepro",'--config_source','ini','--model_id','test001',
#       '-train_sd','20210101','-train_ed','20210103',
#       '-valid_sd','20210104','-train_ed','20210105',
#       '-test_sd','20210106','-test_ed','20210107']
args=['--symbol','BTC', '--execute_mode',"train",
      '--model_id','test001','--model_name','sdnn',
      '--config_source','ini','--config_mode','dev',
      '--user',"Misaka00001", '--source','ipynb',
      '-train_sd','20210101','-train_ed','20210103',
      '-valid_sd','20210104','-valid_ed','20210105',
      '-test_sd','20210106','-test_ed','20210107']

In [ ]:
parser = make_parser()
arg_dict = vars(parser.parse_args(args))

#meta
_id = arg_dict["model_id"]
config_mode = arg_dict["config_mode"].upper()
model_name = arg_dict["model_name"].upper()

# period
train_start = arg_dict["train_start_date"] if "train_start_date" in arg_dict.keys() else None
train_end = arg_dict["train_end_date"] if "train_end_date" in arg_dict.keys() else None
valid_start = arg_dict["valid_start_date"] if "valid_start_date" in arg_dict.keys() else None
valid_end = arg_dict["valid_end_date"]if "valid_end_date" in arg_dict.keys() else None
test_start = arg_dict["test_start_date"] if "test_start_date" in arg_dict.keys() else None
test_end = arg_dict["test_end_date"] if "test_end_date" in arg_dict.keys() else None

# sym
sym = arg_dict["symbol"] 

# load all parent modules
fp = featurePreprocess(_id)
fp.load_general_config(source="ini", path=None,mode=config_mode)
fp.load_model_config(source="ini", path=None,model_name=model_name)

    
if arg_dict["execute_mode"] == "prepro":
    preprocessing(fp, sym, train_start, train_end, valid_start, valid_end, test_start, test_end)
elif arg_dict["execute_mode"] == "train":
    mlflow_tags = {
        "user":arg_dict["user"],
        "source":arg_dict["source"],
        "run_name": f"TRAIN_{dt.now().strftime('%y%m%d%H%M%s')}"
    }
    le = LearningEvaluator(_id, mlflow_tags)
    le.get_device()
    le.load_general_config(source="ini", path=None,mode=config_mode)
    le.load_model_config(source="ini", path=None, model_name=model_name)
    le.load_model_hparameters(model_name)
    train(fp, le)
elif arg_dict["execute_mode"] == "valid":
    mlflow_tags = {
        "user":arg_dict["user"],
        "source":arg_dict["source"],
        "run_name": f"VALID_{dt.now().strftime('%y%m%d%H%M%s')}"
    }
    le = LearningEvaluator(_id, mlflow_tags)
    le.get_device()
    le.load_general_config(source="ini", path=None,mode=config_mode)
    le.load_model_config(source="ini", path=None, model_name=model_name)
    le.load_model_hparameters(model_name)
    valid(fp, le)
else:
    pass

In [ ]:
from mlflow.entities import ViewType
#serch runs
# query = "params.model = 'CNN' and params.layers = '10' and metrics.`prediction accuracy` >= 0.945"
query = None
run_info = le.mlwriter.client.search_runs(experiment_ids=["1"], filter_string=query, run_view_type=ViewType.ACTIVE_ONLY)[0]
uri =  os.path.join(run_info.info.artifact_uri, "model" ) 
mlflow.pytorch.load_model(uri)

In [ ]:
model_info = le.mlwriter.client.get_latest_versions("test_model")[0]
mlflow.pytorch.load_model(model_info.source)

In [ ]:
model_info.source

In [ ]:
model_uri